# LEO Link Budget

Compute forward (ground-to-space) and return (space-to-ground) C/N₀ and link margin for LEO altitudes from 2 000 km down to 500 km.


In [ ]:
import astropy.units as u
from spacelink.core.noise import noise_power_density
from spacelink.core.path import free_space_path_loss
from spacelink.core.antenna import dish_gain
from spacelink.phy.registry import Registry
from spacelink.phy.performance import ErrorMetric

In [ ]:
modcod_registry = Registry()
modcod_registry.load()

# Concatenated convolutional 1/2 + Reed Solomon (255,223) with I=5 interleaver depth
modcod_id = "CCSDS_TM_RS255223_I5_CONV12_BPSK"
modcod_mode = modcod_registry.modes[modcod_id]
modcod_perf = modcod_registry.get_performance(modcod_id, ErrorMetric.WER)

In [ ]:
distance = 2000 * u.km
frequency = 8.4 * u.GHz
symbol_rate = 1 * u.MHz
codeword_error_rate = 1e-4 * u.dimensionless

tx_power = 10.0 * u.dBW
tx_antenna_gain = dish_gain(0.7 * u.m, frequency, 0.65 * u.dimensionless)
rx_antenna_gain = -10.0 * u.dB

rx_antenna_noise_temp = 50 * u.K
rx_system_noise_temp = 100 * u.K
total_noise_temp = rx_antenna_noise_temp + rx_system_noise_temp

eirp = tx_power + tx_antenna_gain

required_ebn0 = modcod_perf.error_rate_to_ebno(codeword_error_rate)
bit_rate = modcod_mode.info_bit_rate(symbol_rate)

print(f"Transmit Gain: {tx_antenna_gain:0.2f}")
print(f"Receive Gain: {rx_antenna_gain:0.2f}")
print(f"EIRP: {eirp:0.2f}")
print(f"Required Eb/N0: {required_ebn0:0.2f}")
print(f"Info Bit Rate: {bit_rate.to(u.kHz).value:0.2f} kbps")

In [ ]:
# Note here that the units can be in km, GHz, whatever
# They will automatically get converted to fundamental units for the computation
path_loss = free_space_path_loss(distance, frequency)
rx_power = eirp - path_loss + rx_antenna_gain
rx_noise_power_density = noise_power_density(total_noise_temp).to("dB(W/Hz)")
cn0 = rx_power - rx_noise_power_density
print(f"C/N0: {cn0:0.2f}")

In [ ]:
ebn0 = cn0 - bit_rate.to(u.dBHz)
margin = ebn0 - required_ebn0
print(f"Eb/N0: {ebn0:0.2f}")
print(f"Margin: {margin:0.2f}")